In [22]:
import sys
sys.path.insert(0, "../..")
import config as cfg
import gc

In [23]:
import pandas as pd
import numpy as np
from metrics import *
from helper import *

In [24]:
train = pd.read_pickle(cfg.PREPARED_TRAIN_DATA_PATH)
cat_test = pd.read_pickle(os.path.join(cfg.TEST_PRED_PATH, 'catboost', 'baseline.pkl'))
stack_test = pd.read_pickle(os.path.join(cfg.TEST_PRED_PATH, 'stacking', 'baseline.pkl'))
cat_oof = pd.read_pickle(os.path.join(cfg.OOF_PRED_PATH, 'catboost', 'baseline.pkl'))
stack_oof = pd.read_pickle(os.path.join(cfg.OOF_PRED_PATH, 'stacking', 'baseline.pkl'))

In [25]:
EXPERIMENT_FAMILY_NAME = 'blending'
EXPERIMENT_NAME = 'baseline'

In [26]:
stack

,heart_failure,angina_pectoris_ischemic_heart_disease_myocardial_infarction,other_heart_diseases,stroke,arterial_hypertension
id,,,,,
54-001-019-01,0.510204,0.524333,0.527473,0.507064,0.508634
54-002-133-01,0.832025,0.832025,0.836735,0.800628,0.822606
54-001-007-01,0.591837,0.598116,0.543171,0.588697,0.566719
54-102-116-01,0.098901,0.122449,0.102041,0.125589,0.108320
54-502-005-02,0.824176,0.814757,0.791209,0.891680,0.795918
...,...,...,...,...,...
54-102-095-01,0.750392,0.755102,0.781790,0.762951,0.788069
54-102-235-01,0.772370,0.789639,0.742543,0.828885,0.784929
54-502-016-01,0.868132,0.847724,0.869702,0.830455,0.843014


In [27]:
cat_test = cat_test.rank() / (len(cat_test) - 1)
stack_test = stack_test.rank() / (len(stack_test) - 1)
blend_test = (stack_test + cat_test) / 2

cat_oof = cat_oof.rank() / (len(cat_oof) - 1)
stack_oof = stack_oof.rank() / (len(stack_oof) - 1)
blend_oof = (cat_oof + stack_oof) / 2

In [28]:
tresholds = get_tresholds(train[cfg.TARGETS], blend_oof)
sample_submission = pd.read_csv(cfg.SAMPLE_SUBMISSION_PATH).set_index('ID')
submission = make_prediction(blend_test, tresholds, sample_submission)

0.6887941440830042


In [30]:
submission_path = os.path.join(cfg.SUBMISSION_PATH, EXPERIMENT_FAMILY_NAME)
check_path(submission_path)
submission.to_csv(os.path.join(submission_path, f'{EXPERIMENT_NAME}.csv'))

blend_oof_path = os.path.join(cfg.OOF_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(blend_oof_path)
blend_oof.to_pickle(os.path.join(pred_proba_oof_path, f'{EXPERIMENT_NAME}.pkl'))

blend_test_path = os.path.join(cfg.TEST_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(blend_test_path)
blend_test.to_pickle(os.path.join(blend_test_path, f'{EXPERIMENT_NAME}.pkl'))